# PIP model 🐾 🤖
In this notebook I import and play with my PIP model, designed in blender

In [1]:
!pip install pybullet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 6.8 MB/s eta 0:00:00


In [2]:
import pybullet as p
import pybullet_data
import numpy as np
import os

# Connect to PyBullet
p.connect(p.DIRECT)

# Set additional search path for PyBullet
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Load a plane for the simulation
p.loadURDF("plane.urdf")


0

In [3]:
!git clone https://github.com/Gaianeve/Real_Mega_Fufi.git

Cloning into 'Real_Mega_Fufi'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 85 (delta 27), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (85/85), 456.39 KiB | 3.90 MiB/s, done.
Resolving deltas: 100% (27/27), done.


## Loading model 📡 🛰

In [4]:
%cd Real_Mega_Fufi/Robot/PIPPA

/content/Real_Mega_Fufi/Robot/PIPPA


In [6]:
PIPPA_id = p.loadURDF("PIPPA.urdf")


## Useful info and tools 🎮 📚 🏮

In [7]:
# Get the number of joints
num_joints = p.getNumJoints(PIPPA_id)
print(f"Number of joints: {num_joints}")

# Get information about each joint
for joint_index in range(num_joints):
    joint_info = p.getJointInfo(PIPPA_id, joint_index)
    joint_name = joint_info[1].decode('UTF-8')  # Decode bytes to string
    joint_type = joint_info[2]  # Joint type (e.g., revolute, spherical)

    print(f"Joint Index: {joint_index}, Name: {joint_name}, Type: {joint_type}")

Number of joints: 18
Joint Index: 0, Name: Bottom_straight_to_1, Type: 4
Joint Index: 1, Name: Bottom_to_Column_1, Type: 4
Joint Index: 2, Name: elastic_to_column_1, Type: 4
Joint Index: 3, Name: Leg_to_elastic_1, Type: 4
Joint Index: 4, Name: Cover_to_leg_1, Type: 4
Joint Index: 5, Name: Cover_to_top_1, Type: 4
Joint Index: 6, Name: Bottom_straight_to__3, Type: 4
Joint Index: 7, Name: Bottom_to_column3, Type: 4
Joint Index: 8, Name: elastic_to_column_3, Type: 4
Joint Index: 9, Name: Leg_to_elastic_3, Type: 4
Joint Index: 10, Name: Cover_to_leg_3, Type: 4
Joint Index: 11, Name: Cover_to_top_3, Type: 4
Joint Index: 12, Name: Bottom_stright_to_2, Type: 4
Joint Index: 13, Name: Bottom_to_column_2, Type: 4
Joint Index: 14, Name: elastic_to_column_2, Type: 4
Joint Index: 15, Name: Leg_to_elastic_2, Type: 4
Joint Index: 16, Name: Cover_to_leg_2, Type: 4
Joint Index: 17, Name: Cover_to_top_2, Type: 4


In [ ]:
# AISI 304 (Acciaio Inox)
p.changeDynamics(robot_id, link_index_AISI304, lateralFriction=0.6, restitution=0.1, rollingFriction=0.002, spinningFriction=0.01)

# AL 6082 (Alluminio)
p.changeDynamics(robot_id, link_index_AL6082, lateralFriction=0.3, restitution=0.05, rollingFriction=0.001, spinningFriction=0.005)

In [ ]:
# Esempio di aumentare la rigidità e il damping del contatto per simulare un materiale molto rigido
p.changeDynamics(
    bodyUniqueId=robot_id,
    linkIndex=0,  # L'indice del link che vuoi configurare
    mass=10.0,  # Imposta una massa realistica per l'acciaio maraging
    lateralFriction=0.9,  # Aumenta per materiali con alto attrito
    restitution=0.1,  # L'acciaio ha un coefficiente di restituzione basso
    contactStiffness=1e6,  # Simula l'alta rigidità dell'acciaio maraging
    contactDamping=1e3  # Smorzamento moderato per dissipare energia
)


## Physics parameters 🎈 🔮

1.  `lateralFriction=0.9` 🐑

  *   **Descrizione**: Rappresenta il coefficiente di attrito laterale tra il link e la superficie su cui entra in contatto.
  *   **Effetto:**  Più alto è il valore, maggiore sarà l'attrito tra il link e la superficie, il che significa che il link resisterà di più allo scivolamento lungo la superficie. Un valore di 1.0 o vicino a esso significa un attrito alto, mentre un valore basso (come 0.1) indica poco attrito, e il link scivolerà più facilmente.

  *   **Esempio:** Materiali ruvidi o che non scivolano facilmente, come gomma o superfici ruvide, hanno un coefficiente di attrito più alto. Materiali come ghiaccio o superfici lisce e scivolose avranno un coefficiente basso.

*Caso Acciaio*: Il valore 0.9 è abbastanza alto, riflettendo l'alto attrito dell'acciaio maraging a contatto con superfici ruvide o altri metalli.


2.  `restitution=0.1` 🐬

  *   **Descrizione**: Rappresenta il coefficiente di restituzione, che determina quanto "rimbalza" un oggetto dopo un impatto.

  *   **Effetto:**  Un valore di 1.0 significa che l'oggetto rimbalza completamente (energia conservata), mentre un valore di 0.0 significa che non c'è rimbalzo (l'energia viene completamente assorbita).

  *   **Esempio:** Materiali elastici come la gomma hanno un alto coefficiente di restituzione (vicino a 1.0), mentre materiali rigidi e non elastici come il piombo o l'acciaio hanno un coefficiente basso.

*Caso Acciaio*: L'acciaio ha un basso coefficiente di restituzione (intorno a 0.1), il che significa che non rimbalza molto dopo un impatto, riflettendo la sua natura rigida e non elastica.


3.  `contactStiffness=1e6` 🐨

  *   **Descrizione**: Rappresenta la rigidità del contatto, o quanto è resistente il materiale alla compressione quando viene a contatto con un'altra superficie.

  *   **Effetto:**  Un valore più alto significa che il materiale è molto rigido e si deformerà pochissimo durante un contatto (simulando l'effetto di un modulo di Young alto), mentre un valore basso rappresenta materiali più morbidi che si deformano più facilmente.

  *   **Esempio:** Materiali come il gommapiuma hanno una bassa rigidità del contatto, mentre materiali molto rigidi come acciaio o ceramica hanno un'alta rigidità.

*Caso Acciaio*: Un valore come 1e6 simula una rigidezza molto alta, tipica dell'acciaio maraging, che si deforma molto poco sotto pressione.


4.  `contactDamping=1e3` 🦘
  
  *   **Descrizione**: Rappresenta lo smorzamento del contatto, che controlla quanto velocemente l'energia del contatto viene dissipata nel tempo.

  *   **Effetto:** Un valore più alto significa che il contatto perde energia rapidamente, smorzando le vibrazioni o il rimbalzo. Un valore basso implica che il sistema è più elastico e manterrà energia più a lungo.

  *   **Esempio:** Un materiale molto elastico, come una molla ben tesa, ha uno smorzamento basso, il che significa che rimbalza e vibra a lungo. Un materiale con alto smorzamento, come l'acciaio, disperde rapidamente energia durante un impatto, quindi non rimbalza molto e smorza le vibrazioni.

*Caso Acciaio*: Il valore 1e3 rappresenta uno smorzamento moderato, coerente con il comportamento dell'acciaio maraging, che dissipa energia abbastanza rapidamente senza rimbalzi significativi.


